In [21]:
! pip uninstall keras -y
! pip uninstall keras-nightly -y
! pip uninstall tf-keras -y


In [22]:
! pip install tensorflow==2.15.0
! pip install scikeras scikit-learn pandas numpy


Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires keras>=3.2.0, but you have keras 2.15.0 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-3.13.1-py3-none-any.whl.metadata (6.3 kB)
Using cached keras-3.13.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.13.1 which is incompatible.


In [23]:
import pandas as pd
import numpy as np
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from scikeras.wrappers import KerasClassifier


In [24]:
data = pd.read_csv("Churn_Modelling.csv")

data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

# Label Encoding Gender
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

# One Hot Encoding Geography
one_hot_encoder_geo = OneHotEncoder(sparse_output=False)
geo = one_hot_encoder_geo.fit_transform(data[['Geography']])

data.drop(['Geography'], axis=1, inplace=True)

data = pd.concat(
    [data, pd.DataFrame(
        geo,
        columns=one_hot_encoder_geo.get_feature_names_out(['Geography'])
    )],
    axis=1
)

# Split features / target
X = data.drop('Exited', axis=1)
y = data['Exited']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# Scaling
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Save encoders & scaler
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('one_hot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(one_hot_encoder_geo, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [25]:
def create_model(neurons=32, layers=1, learning_rate=0.001):
    tf.keras.backend.clear_session()

    model = Sequential()
    model.add(Dense(neurons, activation="relu",
                    input_shape=(x_train.shape[1],)))

    for _ in range(layers - 1):
        model.add(Dense(neurons, activation="relu"))

    model.add(Dense(1, activation="sigmoid"))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [26]:
clf = KerasClassifier(
    model=create_model,
    epochs=50,
    batch_size=32,
    verbose=0
)


In [27]:
param_grid = {
    "model__neurons": [16, 32, 64],
    "model__layers": [1, 2, 3],
    "model__learning_rate": [0.001, 0.01],
    "batch_size": [16, 32],
    "epochs": [30, 50]
}


In [ ]:
grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=3,          # use 3, not 5 (ANN is expensive)
    n_jobs=1,
    scoring="accuracy",
    verbose=1
)

grid_result = grid.fit(x_train, y_train)
print("Best Accuracy:", grid_result.best_score_)
print("Best Parameters:", grid_result.best_params_)


Fitting 3 folds for each of 72 candidates, totalling 216 fits



C:\Users\KIIT0001\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\KIIT0001\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\KIIT0001\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
 

Best Accuracy: 0.8606246049701403
Best Parameters: {'batch_size': 32, 'epochs': 30, 'model__layers': 1, 'model__learning_rate': 0.01, 'model__neurons': 32}


: 